In [1]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int16',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float64', # was 'float32'
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32', # was 'float16'
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float32', # was 'float16'
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float64', # was 'float32'
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float64', # was 'float32'
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32', # was 'float16'
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float64', # was 'float32'
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [9]:
# import the processed dataset
with open(r"C:\Users\admin\Desktop\Data Science UWR\Machine Learning\MLWR\clean_data\cleanDF.pickle", "rb") as input_file:
    malware = pickle.load(input_file)

In [41]:
# import the original dataset (for comparison purposes)
malware_original = pd.read_csv('train.csv', dtype = dtypes)

In [46]:
# get a list of features with missing data in the original dataset
missing_series = (malware.isnull().sum()/malware.shape[0]).sort_values(ascending = False)
cols_with_missing_data = list(missing_series[missing_series != 0].index)

In [79]:
# import pickles with the conditinal MIs etc.
with open(r"C:\Users\admin\Desktop\Data Science UWR\Machine Learning\MLWR\mutualInfos.pickle", "rb") as input_file:
    MI_target_feat = pickle.load(input_file)

Considering what we have done so far regarding the choice of the appropriate features, it makes sense to draw conclusions from our analysis. Below I have included a couple of (I believe) non-controversial candidates to get rid off based on the analysis we have conducted so far. Comments/suggestions/pointingg out mistakes is more than encouraged.

- **PuaMode** - the 'baddest apple' of the whole dataset, >99.97% data missing. I guess it is safe to assume that we will be more than fine without it? Interestingly, the conditional mutual information selection that Rodrigo did had **PuaMode** in its top 15 features?

In [12]:
malware['PuaMode'].value_counts()

NaN      8919174
on          2307
audit          2
Name: PuaMode, dtype: int64

- **Census_ProcessorClass** - the second 'baddest apple', >99.58% missing values. Might as well get rid of it straightaway? Even the description seems to agree: 'No longer maintained and updated'

In [13]:
malware['Census_ProcessorClass'].value_counts()

NaN     8884852
mid       20914
low        9621
high       6096
Name: Census_ProcessorClass, dtype: int64

Furthermore, I believe that the class of the processor is more 'fully' described by the **Census_ProcessorModelIdentifier** feature? Moreover it has NO missing data

In [47]:
print('Census_ProcessorModelIdentifier' in cols_with_missing_data)
malware['Census_ProcessorModelIdentifier'].value_counts()

False


2697    330626
1998    267397
2660    191392
2373    175407
1992    171728
         ...  
1750         1
1748         1
1745         1
1744         1
2            1
Name: Census_ProcessorModelIdentifier, Length: 3428, dtype: int64

- **Processor** - yet another processor-related feature which I think can be dropped. Afterall, it is another way of classifying the processor, which again seems reduntant if we consider **Census_ProcessorModelIdentifier** giving the exact models...

- **Census_OSArchitecture** - this is literally the exact same feature as **Processor**. The numbers in value_counts() don't match exactly most probably due to non-matching number of missing data in both columns.

In [152]:
print(malware['Processor'].value_counts())
print("\n")
print(malware['Census_OSArchitecture'].value_counts())

x64      8105435
x86       815702
arm64        346
NaN            0
Name: Processor, dtype: int64


amd64    8105885
x86       815252
arm64        346
NaN            0
Name: Census_OSArchitecture, dtype: int64


- **DefaultBrowsersIdentifier** - >95% missing. A lot, enough to bin it? Let the readers decide. Apparently, there were more than 2000 different browsers, I had no idea that there exist that many of them.

- **Census_IsFlightingInternal** - There is no description of this feature available. Also, I think that Daniel's treatment changed all the NaN values in this binary feature into zeros, and it had >83% missing features. My guess is that this may not be far from the real case here (only 21 rows of different class)

In [24]:
malware['Census_IsFlightingInternal'].value_counts()

0    8921462
1         21
Name: Census_IsFlightingInternal, dtype: int64

In [89]:
# bias in target if Census_IsFlightingInternal == 1 ?
malware[(malware['Census_IsFlightingInternal'] == 1)]['HasDetections'].value_counts(normalize = True)
# seems 'balanced'?

1    0.619048
0    0.380952
Name: HasDetections, dtype: float64

In [82]:
MI_target_feat['Census_IsFlightingInternal']

8.77867941733168e-05

- **Census_IsFlightsDisabled** - I guess it relates to the above?

In [140]:
malware['Census_IsFlightsDisabled'].value_counts()

0    8921395
1         88
Name: Census_IsFlightsDisabled, dtype: int64

In [93]:
# bias in target if Census_IsFlightsDisabled == 1 ?
malware[(malware['Census_IsFlightsDisabled'] == 1)]['HasDetections'].value_counts(normalize = True)
# seems IMBALANCED

0    0.943182
1    0.056818
Name: HasDetections, dtype: float64

- **Census_IsWIMBootEnabled** - no idea what WIM is, but the situation is analogous to the above but even more severe - there is only 1 WIM Boot enabled. We can't be that unlucky? Also there is >65% data missing.

In [87]:
malware['Census_IsWIMBootEnabled'].value_counts()

0    8921482
1          1
Name: Census_IsWIMBootEnabled, dtype: int64

In [83]:
MI_target_feat['Census_IsWIMBootEnabled']
# the dictionary was developed with the missing data present, this value does NOT correspond
# to the value_counts() from the line above

0.00010287984506374137

- **IsBeta** - I guess beta as in beta version of the 'Defender state information' (Original description. Yes, very vague)?

In [102]:
malware['IsBeta'].value_counts()

0    8921416
1         67
Name: IsBeta, dtype: int64

In [78]:
# bias in target if IsBeta == 1 ?
malware[(malware['IsBeta'] == 1)]['HasDetections'].value_counts(normalize = True)
# seems 'balanced'

0    0.507463
1    0.492537
Name: HasDetections, dtype: float64

- **AutoSampleOptIn** - 'This is the SubmitSamplesConsent value passed in from the service, available on CAMP 9+'. No idea what it is.

In [101]:
malware['AutoSampleOptIn'].value_counts()

0    8921225
1        258
Name: AutoSampleOptIn, dtype: int64

In [96]:
# bias in target if AutoSampleOptIn == 1 ?
malware[(malware['AutoSampleOptIn'] == 1)]['HasDetections'].value_counts(normalize = True)
# seems 'balanced'

1    0.546512
0    0.453488
Name: HasDetections, dtype: float64

- **Census_ThresholdOptIn** - there is no description for that, no idea what the threshold relates to.

In [166]:
malware['Census_ThresholdOptIn'].value_counts()

0    8920667
1        816
Name: Census_ThresholdOptIn, dtype: int64

In [98]:
# bias in target if Census_ThresholdOptIn == 1 ?
malware[(malware['Census_ThresholdOptIn'] == 1)]['HasDetections'].value_counts(normalize = True)
# seems 'balanced'

0    0.531863
1    0.468137
Name: HasDetections, dtype: float64

- **Census_IsPortableOperatingSystem** - 'Indicates whether OS is booted up and running via Windows-To-Go on a USB stick'.

In [154]:
malware['Census_IsPortableOperatingSystem'].value_counts()

0    8916619
1       4864
Name: Census_IsPortableOperatingSystem, dtype: int64

In [100]:
# bias in target if Census_IsPortableOperatingSystem == 1 ?
malware[(malware['Census_IsPortableOperatingSystem'] == 1)]['HasDetections'].value_counts(normalize = True)
# seems 'balanced'

1    0.553248
0    0.446752
Name: HasDetections, dtype: float64

- **Census_DeviceFamily** - 'Indicates the type of device that an edition of the OS is intended for. Example values: Windows.Desktop, Windows.Mobile, and iOS.Phone'. Despite the description, there are only Windows options available.

In [125]:
malware['Census_DeviceFamily'].value_counts()

Windows.Desktop    8907053
Windows.Server       14410
Windows                 20
NaN                      0
Name: Census_DeviceFamily, dtype: int64

In [65]:
# bias in target if Census_DeviceFamily == 1 ?
malware[(malware['Census_DeviceFamily'] == "Windows.Server")]['HasDetections'].value_counts(normalize = True)
# rather IMBALANCED

0    0.650659
1    0.349341
Name: HasDetections, dtype: float64

There are already more than 10 potential features to discard included above, 5 of them with objects as entries - think of all the memory we would be saving...

# My remarks on the results from CMI

In [136]:
CMIs_top_selection = ['Census_SystemVolumeTotalCapacity',
                     'Census_PowerPlatformRoleName',
                     'Census_ChassisTypeName',
                     'Census_InternalBatteryType',
                     'SmartScreen',
                     'AVProductsInstalled',
                     'Processor',
                     'RtpStateBitfield',
                     'OsBuildLab',
                     'AVProductsEnabled',
                     'Census_MDC2FormFactor',
                     'Census_OSArchitecture',
                     'PuaMode',
                     'Census_ProcessorClass',
                     'UacLuaenable',
                     'Census_DeviceFamily',
                     'ProductName',
                     'Platform',
                     'Census_FlightRing',
                     'Census_GenuineStateName',
                     'Census_ProcessorManufacturerIdentifier',
                     'Census_OSWUAutoUpdateOptionsName',
                     'OsPlatformSubRelease',
                     'Census_OSSkuName',
                     'Census_OSEdition',
                     'SkuEdition',
                     'Census_ActivationChannel',
                     'Census_OSBranch',
                     'Census_PrimaryDiskTypeName']

- **Census_ProcessorClass** and **PuaMode** appear in the top list from conditional MIs, which is odd given that they have >99% missing data resulting in columns being pretty much 'constant'

- Both **Processor** and **Census_OSArchitecture** are present - makes sense given they are basically identical

- Reasonable choices are also included, such as **AVProductsInstalled** or **AVProductsEnabled** - that's very comforting

- More to come...

# Possible ideas for further investigation

* can we use only one of the following **SKU**s?

In [168]:
malware['SkuEdition'].value_counts()

Home               5514341
Pro                3224164
Invalid              78054
Education            40694
Enterprise           34357
Enterprise LTSB      20702
Cloud                 5589
Server                3582
NaN                      0
Name: SkuEdition, dtype: int64

In [170]:
malware['Census_OSSkuName'].value_counts()

CORE                            3469869
PROFESSIONAL                    3187913
CORE_SINGLELANGUAGE             1945133
CORE_COUNTRYSPECIFIC             165886
EDUCATION                         40827
ENTERPRISE                        35602
PROFESSIONAL_N                    28522
ENTERPRISE_S                      20022
STANDARD_SERVER                   10128
CLOUD                              6167
CORE_N                             4787
STANDARD_EVALUATION_SERVER         2755
EDUCATION_N                         927
ENTERPRISE_S_N                      881
DATACENTER_EVALUATION_SERVER        829
SB_SOLUTION_SERVER                  684
ENTERPRISE_N                        356
PRO_WORKSTATION                     124
UNLICENSED                           17
DATACENTER_SERVER                    14
PRO_WORKSTATION_N                    12
CLOUDN                                7
PRO_CHINA                             5
ULTIMATE                              4
SERVERRDSH                            4


- More to come...